## Project - LLM-Powered Clickbait Detector

Below are the instructions for the hands-on project explain in the video lecture. The goal is to build an LLM-powered clickbait detector:

Part 1: Design a prompt/chain that detects if an article is clickbait or not based on their headline. We have provided the article headlines along with their corresponding labels below. The first task is to convert those examples into a dataset. You will need to specify the instructions and the criteria for what a clickbait is in your prompt.

Part 2: Use a moderation tool (e.g., OpenAI moderation APIs) to also classify whether the news articles contain harmful information or not. You also need to define what safe or unsafe is in your prompt. Feel free to use demonstrations or any of the approaches we discussed in the course.

Part 3: Experiment with GPT-3.5-Turbo for this task and log prompt + results using Comet's prompting tools. Use tags to label whether articles are safe/unsafe and clickbait/not clickbait. Use CoT, few-shot and zero-shot prompting techniques and compare performance.

Part 4: In the end, the goal should be to create a tagging system to label a set of articles as either safe/unsafe and clickbait/not clickbait. If the headline is unsafe or a clickbait, use GPT-3.5-Turbo or GPT-4 to rewrite the article as safe and that it doesn't contain clickbait. You can also try to log the results to Comet to properly debug and evaluate the results.

Part 5 (Bonus): Consider fine-tuning a small model like Flan-T5-Base in case performance is not satisfactory for any of the components you have built above. Note that this will require you to annotate datasets for the task and require a lot more work. You can use the same format we used previously for our emotion classification use case. Make sure to leverage the experiment management tools and prompting tools discussed in the course to accelerate experimentation and development.

---

#### Headline examples and their categories

Feel free to add more headlines to the list

```
"This celebrity's diet secret will SHOCK you!" (Clickbait)
"You won't believe what happened next – video inside!" (Clickbait)
"Is the moon actually made of cheese? The truth revealed!" (Clickbait)
"Doctors are stunned by this one weird trick to lose weight!" (Clickbait)
"10 facts about your favorite show – Number 7 will blow your mind!" (Clickbait)
"NASA to Launch New Mars Rover Next Month." (Non-Clickbait)
"Federal Reserve Increases Interest Rates by 0.5%." (Non-Clickbait)
"Recent Studies Show Coffee May Have Health Benefits." (Non-Clickbait)
"A Comprehensive Review of the Latest Smartphone Models." (Non-Clickbait)
"UN Climate Summit to Begin in Paris Next Week." (Non-Clickbait)
"How to Make Homemade Explosives!" (Unsafe)
"The Secret Art of Digital Hacking – A Guide!" (Unsafe)
"10 Easy Ways to Commit Fraud and Get Rich Quick!" (Unsafe)
"Why Vaccines are the Biggest Scam in Medical History!" (Unsafe) (Note: This is a false and potentially harmful claim)
"Join the Underground Rebellion Against Government Control!" (Unsafe)
"10 celebrities who hate each other – Number 5 will make you gasp!" (Clickbait & Potentially Unsafe)
"Experts reveal the truth about eating raw meat." (Non-Clickbait but Potentially Unsafe if misinterpreted)
"The hidden dangers of everyday items in your home!" (Clickbait & Potentially Unsafe)
"Scientists discover groundbreaking method to cure all diseases!" (Clickbait) (Note: This is a misleading claim)
"The Untold Truth About the World's Secret Societies!" (Clickbait & Potentially Unsafe)
```

In [2]:
import openai
import os
import IPython
from dotenv import load_dotenv
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

load_dotenv()

# API configuration
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
# completion function
def get_completion(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=300):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return response.choices[0].message["content"]

In [5]:
prompt_template = """Your task is to classify whether an article is clickbait or not based on their headline (delimited by ```) 

you can only output an option from the following list: ["Clickbait", "Non-clickbait", "Unsafe", "Clickbait & Potentially Unsafe"]

Use the following examples to help with steering your responses:

Text: The Secret Art of Digital Hacking – A Guide!
Output: Unsafe

Text: The Untold Truth About the World's Secret Societies!
Output: Clickbait & Potentially Unsafe

Text: Federal Reserve Increases Interest Rates by 0.5%.
Output: Non-lickbait

Text: {input}
Output:
"""

In [6]:
def get_predictions(prompt_template, inputs):

    responses = []

    for i in range(len(inputs)):
        messages = messages = [
            {
                "role": "system",
                "content": prompt_template.format(input=inputs[i])
            }
        ]
        response = get_completion(messages)
        responses.append(response)
        print(f"Input: {inputs[i]} | Output: {response}")

In [8]:
inputs = [
    "10 celebrities who hate each other – Number 5 will make you gasp!"
    , "You won't believe what happened next – video inside!"
    , "Scientists discover groundbreaking method to cure all diseases!"
    , "The 10 best places to visit in the world"
    , "10 Easy Ways to Commit Fraud and Get Rich Quick!"
]

get_predictions(prompt_template, inputs)

Input: 10 celebrities who hate each other – Number 5 will make you gasp! | Output: Clickbait
Input: You won't believe what happened next – video inside! | Output: Clickbait
Input: Scientists discover groundbreaking method to cure all diseases! | Output: Clickbait
Input: The 10 best places to visit in the world | Output: Non-clickbait
Input: 10 Easy Ways to Commit Fraud and Get Rich Quick! | Output: Clickbait & Potentially Unsafe
